In [2]:
import numpy as np
from tensorflow import set_random_seed
from numpy.random import seed
import pandas as pd
import matplotlib
import sys
if sys.version_info[0] >= 3:
    import PySimpleGUI as sg
    import tkinter as Tk
else:
    import PySimpleGUI27 as sg
    import Tkinter as Tk
from matplotlib.backends.backend_tkagg import FigureCanvasAgg
import matplotlib.backends.tkagg as tkagg
import inspect
import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import datetime
matplotlib.use('TkAgg')
from tkinter import *
import tkinter as Tk

In [3]:
#dataset_empty = 0
set_random_seed(1)
seed(1)

In [4]:
#Convert the train data into time series
def time_series_data(data, window, lag):
    dropnan = True
    cols, names = list(), list()

    for i in range(window, 0, -1):
        #past time series data (t-)
        cols.append(data.shift(i))
        names = names + [('%s(t-%d)' % (col, i)) for col in data.columns]
    
    #current time series data (t = 0)
    cols.append(data)
    names = names + [('%s(t)' % (col)) for col in data.columns]
    
    #future data (t + lag)
    cols.append(data.shift(-lag))
    names = names + [('%s(t+%d)' % (col, lag)) for col in data.columns]
    
    #all data
    all_data = pd.concat(cols, axis=1)
    all_data.columns = names
    
    #drops rows with NaN
    if dropnan:
        all_data.dropna(inplace=True)
    return all_data

In [5]:
def get_item_code(item_name):
    items = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\demand\\items.csv', usecols = ['item_code', 'item_name'])
    item_code = items.loc[items['item_name'] == item_name]
    item = item_code.iat[0, 0]
    
    return item

In [6]:
def get_store_loc(store):
    stores = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\demand\\stores_geo.csv', usecols = ['store_code', 'location', 'geo_code'])
    store_loc = stores.loc[stores['store_code'] == store]
    location = store_loc.iat[0, 1]
    
    return location

In [7]:
store = 0
item = 0
item_name = ''
store_loc = ''

def get_result_by_item():
    try:
        
        ########################################### sales data ##########################################################
        dataset = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\sales data-set.csv', usecols = ['Store', 'Item', 'Date', 'Weekly_Sales'])
        train = dataset[['Store', 'Item', 'Date', 'Weekly_Sales']]

        train = train.loc[(train['Store'] == store) & (train['Item'] == item)]
        div = 10000   #weekly sales in 10 thousand dollars
        file_name = "C:\\Users\\mrura\\Downloads\\MRP Final\\sales\\saved nn weights\\by item\\cnn_lstm_weights_s" + str(store) + "_i" + str(item) + ".hdf5"

        #Re-arranges the train dataset to apply shift methods
        train_r = train.sort_values('Date').groupby(['Item', 'Store', 'Date'], as_index=False)
        train_r = train_r.agg({'Weekly_Sales':['mean']})
        train_r.columns = ['Item', 'Store', 'Date', 'Weekly_Sales']
        #train_r = train_r.groupby(['Date'], as_index=False)['Weekly_Sales'].sum()
        train_r['Weekly_Sales'] = train_r['Weekly_Sales'] / div

        #the model will use last 117 weekly sales data and 
        #current timestep (7 days) to forecast next weekly sales data 12 weeks ahead

        window = 117
        lag = 12

        series_data = time_series_data(train_r, window, lag)
        future_dates = series_data[['Date(t+%d)' % lag]]

        if((store == "all") | (store == "ALL") | (store == "All")):
            #drops Item and Store columns
            cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
            for i in range(window, 0, -1):
                cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

            series_data.drop(cols_to_drop, axis=1, inplace=True)
            series_data.drop(['Date(t)'], axis=1, inplace=True)

            lbls_col = 'Weekly_Sales(t+%d)' % lag
            lbls = series_data[lbls_col]
            series_data = series_data.drop(lbls_col, axis=1)
        else:
            if(item >= 0): 
                #drops last record of (t + lag)
                last_record_item = 'Item(t-%d)' % window
                last_record_store = 'Store(t-%d)' % window
                series_data = series_data[(series_data['Item(t)'] == series_data[last_record_item])]
                series_data = series_data[(series_data['Store(t)'] == series_data[last_record_store])]

                #drops Item and Store columns
                cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Item', 'Store', 'Date']]
                for i in range(window, 0, -1):
                    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Item', 'Store', 'Date']]

                series_data.drop(cols_to_drop, axis=1, inplace=True)
                series_data.drop(['Item(t)', 'Store(t)', 'Date(t)'], axis=1, inplace=True)

                lbls_col = 'Weekly_Sales(t+%d)' % lag
                lbls = series_data[lbls_col]
                series_data = series_data.drop(lbls_col, axis=1)
            else:
                #drops Item and Store columns
                cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
                for i in range(window, 0, -1):
                    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

                series_data.drop(cols_to_drop, axis=1, inplace=True)
                series_data.drop(['Date(t)'], axis=1, inplace=True)

                lbls_col = 'Weekly_Sales(t+%d)' % lag
                lbls = series_data[lbls_col]
                series_data = series_data.drop(lbls_col, axis=1)

        X_train, X_test, Y_train, Y_test = train_test_split(series_data, lbls.values, test_size=0.3, random_state=0, shuffle=False)
        X_test = series_data     
        X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

        subsequences = 2
        time_steps = X_test_series.shape[1] // subsequences
        X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, time_steps, 1))
        X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, time_steps, 1))

        epochs = 2000
        batch = 32
        learning_rate = 0.00000001
        adam = optimizers.Adam(learning_rate)

        model_cnn_lstm = Sequential()
        model_cnn_lstm.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape = (None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
        model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model_cnn_lstm.add(TimeDistributed(Flatten()))
        model_cnn_lstm.add(LSTM(60, activation = 'relu'))
        model_cnn_lstm.add(Dropout(0.1))
        model_cnn_lstm.add(Dense(30, activation = 'relu'))
        model_cnn_lstm.add(Dropout(0.1))
        model_cnn_lstm.add(Dense(1))
        model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

        model_cnn_lstm.load_weights(file_name)
        model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

        #prediction
        #cnn_lstm_train_prediction = model_cnn_lstm.predict(X_train_series_sub)
        cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub)

        test_p = []
        for i in range(0, cnn_lstm_test_prediction.shape[0]):
            test_p.append(cnn_lstm_test_prediction[i][0])

        predictions = pd.DataFrame({'Date' : list(future_dates['Date(t+12)']), 'Weekly_Sales' : test_p})

        train_p = train_r
        actual_data = train_p.tail(117 + 12)
        train_p.drop(['Item', 'Store'], axis=1, inplace=True)
        actual_data.drop(['Item', 'Store'], axis=1, inplace=True)

        for k in range(0, 12):
            X_test_predict = train_p.tail(117 + 12)    
            max_date = X_test_predict['Date'].max()
            next_date = datetime.datetime.strptime(max_date, '%Y-%m-%d').date() + timedelta(days=7)
            df = pd.DataFrame({'Date' : next_date.strftime("%Y-%m-%d"), 
                               'Weekly_Sales' : [0]})
            X_test_predict = X_test_predict.append(df)
            train_p = train_p.append(df)
            predictions = predictions.append(df)
            actual_data = actual_data.append(df)

            X_test_predict = X_test_predict.reset_index(drop = True)
            train_p = train_p.reset_index(drop = True)
            predictions = predictions.reset_index(drop = True)
            actual_data = actual_data.reset_index(drop = True)

            series_data_p = time_series_data(X_test_predict, window, lag)

            cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
            for i in range(window, 0, -1):
                cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

            series_data_p.drop(cols_to_drop, axis=1, inplace=True)
            series_data_p.drop(['Date(t)'], axis=1, inplace=True)

            lbls_col = 'Weekly_Sales(t+%d)' % lag
            lbls = series_data_p[lbls_col]
            series_data_p = series_data_p.drop(lbls_col, axis=1)

            X_test_series_p = series_data_p.values.reshape((series_data_p.shape[0], X_test.shape[1], 1))

            subsequences = 2
            time_steps_p = X_test_series_p.shape[1] // subsequences
            X_test_series_sub_p = X_test_series_p.reshape((X_test_series_p.shape[0], subsequences, time_steps_p, 1))   

            cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub_p)

            train_p.loc[(train_p['Date'] == next_date.strftime("%Y-%m-%d"))  & (train_p['Weekly_Sales'] == 0), ['Weekly_Sales']] = cnn_lstm_test_prediction[0][0]
            predictions.loc[(predictions['Date'] == next_date.strftime("%Y-%m-%d")) & (predictions['Weekly_Sales'] == 0), ['Weekly_Sales']] = cnn_lstm_test_prediction[0][0]

        graph_data = actual_data.merge(predictions, left_on = ['Date'], right_on = ['Date'], how = 'outer')
        graph_data = graph_data.loc[graph_data['Date'] >= '2012-07-29']
        graph_data.loc[graph_data['Weekly_Sales_x'] == 0, ['Weekly_Sales_x']] = np.nan #Weekly_Sales_x is actual data and Weekly_Sales_y is the predicted data

        plt.subplots(figsize=(15, 6))
        plt.subplot(221)
        plt.plot(graph_data.Date, graph_data.Weekly_Sales_x, color='blue')
        plt.plot(graph_data.Date, graph_data.Weekly_Sales_y, dashes=[10, 5, 10, 5], color='red')
        plt.legend(['Actual Sales Data', 'Predicted Sales Data'], loc='upper right')
        plt.yticks(np.arange(0, 10, 2))
        plt.xticks(graph_data.Date, rotation=90)
        plt.title('Weekly sales and forecasts of ' + str(item_name) + ' in Store-' + str(store) + ', Location-' + str(store_loc))
        plt.xlabel('Date (yyyy-mm-dd)')
        plt.ylabel('Weekly Sales (in 10 million dollars)')
        plt.grid(True)

        if((store > 0) & (item > 0)):
            ########################################### demand data ##########################################################
            dataset = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\demand\\interest_over_time_s' + str(store) + '.csv', usecols = ['Store', 'Item', 'Date', 'Weekly_Demand'])
            train = dataset[['Store', 'Item', 'Date', 'Weekly_Demand']]

            train = train.loc[(train['Store'] == store) & (train['Item'] == item)]
            div = 10  
            file_name = "C:\\Users\\mrura\\Downloads\\MRP Final\\demand\\saved nn weights\\cnn_lstm_gtrends_weights_s" + str(store) + "_i" + str(item) + ".hdf5"

            #Re-arranges the train dataset to apply shift methods
            train_r = train.sort_values('Date').groupby(['Item', 'Store', 'Date'], as_index=False)
            train_r = train_r.agg({'Weekly_Demand':['mean']})
            train_r.columns = ['Item', 'Store', 'Date', 'Weekly_Demand']
            #train_r = train_r.groupby(['Date'], as_index=False)['Weekly_Demand'].sum()
            train_r['Weekly_Demand'] = train_r['Weekly_Demand'] / div    #weekly demand data in 0-10 scale

            #the model will use last 117 weekly sales data and 
            #current timestep (7 days) to forecast next weekly sales data 12 weeks ahead

            window = 117
            lag = 12

            series_data = time_series_data(train_r, window, lag)
            future_dates = series_data[['Date(t+%d)' % lag]]

            if (item >= 4): 
                #drops last record of (t + lag)
                last_record_item = 'Item(t-%d)' % window
                last_record_store = 'Store(t-%d)' % window
                series_data = series_data[(series_data['Item(t)'] == series_data[last_record_item])]
                series_data = series_data[(series_data['Store(t)'] == series_data[last_record_store])]

                #drops Item and Store columns
                cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Item', 'Store', 'Date']]
                for i in range(window, 0, -1):
                    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Item', 'Store', 'Date']]

                series_data.drop(cols_to_drop, axis=1, inplace=True)
                series_data.drop(['Item(t)', 'Store(t)', 'Date(t)'], axis=1, inplace=True)

                lbls_col = 'Weekly_Demand(t+%d)' % lag
                lbls = series_data[lbls_col]
                series_data = series_data.drop(lbls_col, axis=1)
            else:
                #drops Item and Store columns
                cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
                for i in range(window, 0, -1):
                    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

                series_data.drop(cols_to_drop, axis=1, inplace=True)
                series_data.drop(['Date(t)'], axis=1, inplace=True)

                lbls_col = 'Weekly_Demand(t+%d)' % lag
                lbls = series_data[lbls_col]
                series_data = series_data.drop(lbls_col, axis=1)


            #if(series_data.shape[1] < 143):
            X_train, X_test, Y_train, Y_test = train_test_split(series_data, lbls.values, test_size=0.3, random_state=0, shuffle=False)
            X_test = series_data     
            X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
            X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

            subsequences = 2
            time_steps = X_test_series.shape[1] // subsequences
            X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, time_steps, 1))
            X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, time_steps, 1))

            epochs = 5000
            batch = 32
            learning_rate = 0.00000001
            adam = optimizers.Adam(learning_rate)

            model_cnn_lstm = Sequential()
            model_cnn_lstm.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape = (None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
            model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
            model_cnn_lstm.add(TimeDistributed(Flatten()))
            model_cnn_lstm.add(LSTM(60, activation = 'relu'))
            model_cnn_lstm.add(Dropout(0.5))
            model_cnn_lstm.add(Dense(30, activation = 'relu'))
            model_cnn_lstm.add(Dropout(0.5))
            model_cnn_lstm.add(Dense(1))
            model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

            model_cnn_lstm.load_weights(file_name)
            model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

            #prediction
            #cnn_lstm_train_prediction = model_cnn_lstm.predict(X_train_series_sub)
            cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub)

            test_p = []
            for i in range(0, cnn_lstm_test_prediction.shape[0]):
                test_p.append(cnn_lstm_test_prediction[i][0])

            if (item >= 4): 
                predictions = pd.DataFrame({#'Item' : 1,
                                            #'Store' : 1,
                                            'Date' : list(future_dates['Date(t+12)']),
                                            'Weekly_Demand' : test_p})
            else:
                predictions = pd.DataFrame({'Item' : 1,
                                            'Store' : 1,
                                            'Date' : list(future_dates['Date(t+12)']),
                                            'Weekly_Demand' : test_p})

            train_p = train_r
            actual_data = train_p.tail(117 + 12)
            if (item >= 4): 
                train_p.drop(['Item', 'Store'], axis=1, inplace=True)
                actual_data.drop(['Item', 'Store'], axis=1, inplace=True)

            for k in range(0, 12):
                X_test_predict = train_p.tail(117 + 12)    
                max_date = X_test_predict['Date'].max()
                next_date = datetime.datetime.strptime(max_date, '%Y-%m-%d').date() + timedelta(days=7)
                if (item >= 4): 
                    df = pd.DataFrame({#'Item' : 1,
                                        #'Store' : 1,
                                        'Date' : next_date.strftime("%Y-%m-%d"), 
                                        'Weekly_Demand' : [0]})
                else:
                    df = pd.DataFrame({'Item' : 1,
                                        'Store' : 1,
                                        'Date' : next_date.strftime("%Y-%m-%d"), 
                                        'Weekly_Demand' : [0]})

                X_test_predict = X_test_predict.append(df)
                train_p = train_p.append(df)
                predictions = predictions.append(df)
                actual_data = actual_data.append(df)

                X_test_predict = X_test_predict.reset_index(drop = True)
                train_p = train_p.reset_index(drop = True)
                predictions = predictions.reset_index(drop = True)
                actual_data = actual_data.reset_index(drop = True)

                series_data_p = time_series_data(X_test_predict, window, lag)

                cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
                for i in range(window, 0, -1):
                    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

                series_data_p.drop(cols_to_drop, axis=1, inplace=True)
                series_data_p.drop(['Date(t)'], axis=1, inplace=True)

                lbls_col = 'Weekly_Demand(t+%d)' % lag
                lbls = series_data_p[lbls_col]
                series_data_p = series_data_p.drop(lbls_col, axis=1)

                X_test_series_p = series_data_p.values.reshape((series_data_p.shape[0], X_test.shape[1], 1))

                subsequences = 2
                time_steps_p = X_test_series_p.shape[1] // subsequences
                X_test_series_sub_p = X_test_series_p.reshape((X_test_series_p.shape[0], subsequences, time_steps_p, 1))   

                cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub_p)

                train_p.loc[(train_p['Date'] == next_date.strftime("%Y-%m-%d"))  & (train_p['Weekly_Demand'] == 0), ['Weekly_Demand']] = cnn_lstm_test_prediction[0][0]
                predictions.loc[(predictions['Date'] == next_date.strftime("%Y-%m-%d")) & (predictions['Weekly_Demand'] == 0), ['Weekly_Demand']] = cnn_lstm_test_prediction[0][0]

            graph_data = actual_data.merge(predictions, left_on = ['Date'], right_on = ['Date'], how = 'outer')
            graph_data = graph_data.loc[graph_data['Date'] >= '2012-07-29']
            graph_data.loc[graph_data['Weekly_Demand_x'] == 0, ['Weekly_Demand_x']] = np.nan #Weekly_Sales_x is actual data and Weekly_Sales_y is the predicted data

            plt.subplot(222)
            plt.plot(graph_data.Date, graph_data.Weekly_Demand_x, color='green')
            plt.plot(graph_data.Date, graph_data.Weekly_Demand_y, dashes=[10, 5, 10, 5], color='orange')
            plt.legend(['Actual Demand Data', 'Predicted Demand Data'], loc='upper right')
            plt.yticks(np.arange(0, 10, 2))
            plt.xticks(graph_data.Date, rotation=90)
            plt.title('Weekly demand and forecasts of ' + str(item_name) + ' in Store-' + str(store) + ', Location-' + str(store_loc))
            plt.xlabel('Date (yyyy-mm-dd)')
            plt.ylabel('Weekly Demand (in 0-10 scale)')
            plt.grid(True)

        else:
            print("Insufficient data!")
    except:
        print("Insufficient data!")
        
    return plt.gcf()


In [8]:
def get_result_by_store():
    try:
        
        ########################################### sales data ##########################################################
        dataset = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\sales data-set.csv', usecols = ['Store', 'Item', 'Date', 'Weekly_Sales'])
        train = dataset[['Store', 'Item', 'Date', 'Weekly_Sales']]    
        train = train.loc[train['Store'] == store]
        div = 1000000   #weekly sales in 10 million dollars
        file_name = "C:\\Users\\mrura\\Downloads\\MRP Final\\sales\\saved nn weights\\by store\\cnn_lstm_weights_store_" + str(store) + ".hdf5"

        #Re-arranges the train dataset to apply shift methods
        train_r = train.sort_values('Date').groupby(['Item', 'Store', 'Date'], as_index=False)
        train_r = train_r.agg({'Weekly_Sales':['mean']})
        train_r.columns = ['Item', 'Store', 'Date', 'Weekly_Sales']
        train_r = train_r.groupby(['Date'], as_index=False)['Weekly_Sales'].sum()
        train_r['Weekly_Sales'] = train_r['Weekly_Sales'] / div

        #the model will use last 117 weekly sales data and 
        #current timestep (7 days) to forecast next weekly sales data 12 weeks ahead

        window = 117
        lag = 12

        series_data = time_series_data(train_r, window, lag)
        future_dates = series_data[['Date(t+%d)' % lag]]

        #drops Item and Store columns
        cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
        for i in range(window, 0, -1):
            cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

        series_data.drop(cols_to_drop, axis=1, inplace=True)
        series_data.drop(['Date(t)'], axis=1, inplace=True)

        lbls_col = 'Weekly_Sales(t+%d)' % lag
        lbls = series_data[lbls_col]
        series_data = series_data.drop(lbls_col, axis=1)

        X_train, X_test, Y_train, Y_test = train_test_split(series_data, lbls.values, test_size=0.3, random_state=0, shuffle=False)
        X_test = series_data     
        X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

        subsequences = 2
        time_steps = X_test_series.shape[1] // subsequences
        X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, time_steps, 1))
        X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, time_steps, 1))

        epochs = 2000
        batch = 32
        learning_rate = 0.00000001
        adam = optimizers.Adam(learning_rate)

        model_cnn_lstm = Sequential()
        model_cnn_lstm.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape = (None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
        model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model_cnn_lstm.add(TimeDistributed(Flatten()))
        model_cnn_lstm.add(LSTM(60, activation = 'relu'))
        model_cnn_lstm.add(Dropout(0.1))
        model_cnn_lstm.add(Dense(30, activation = 'relu'))
        model_cnn_lstm.add(Dropout(0.1))
        model_cnn_lstm.add(Dense(1))
        model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

        model_cnn_lstm.load_weights(file_name)
        model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

        #prediction
        #cnn_lstm_train_prediction = model_cnn_lstm.predict(X_train_series_sub)
        cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub)

        test_p = []
        for i in range(0, cnn_lstm_test_prediction.shape[0]):
            test_p.append(cnn_lstm_test_prediction[i][0])

        predictions = pd.DataFrame({'Date' : list(future_dates['Date(t+12)']), 'Weekly_Sales' : test_p})

        train_p = train_r
        actual_data = train_p.tail(117 + 12)
        #train_p.drop(['Item', 'Store'], axis=1, inplace=True)
        #actual_data.drop(['Item', 'Store'], axis=1, inplace=True)

        for k in range(0, 12):
            X_test_predict = train_p.tail(117 + 12)    
            max_date = X_test_predict['Date'].max()
            next_date = datetime.datetime.strptime(max_date, '%Y-%m-%d').date() + timedelta(days=7)
            df = pd.DataFrame({'Date' : next_date.strftime("%Y-%m-%d"), 
                               'Weekly_Sales' : [0]})
            X_test_predict = X_test_predict.append(df)
            train_p = train_p.append(df)
            predictions = predictions.append(df)
            actual_data = actual_data.append(df)

            X_test_predict = X_test_predict.reset_index(drop = True)
            train_p = train_p.reset_index(drop = True)
            predictions = predictions.reset_index(drop = True)
            actual_data = actual_data.reset_index(drop = True)

            series_data_p = time_series_data(X_test_predict, window, lag)

            cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
            for i in range(window, 0, -1):
                cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

            series_data_p.drop(cols_to_drop, axis=1, inplace=True)
            series_data_p.drop(['Date(t)'], axis=1, inplace=True)

            lbls_col = 'Weekly_Sales(t+%d)' % lag
            lbls = series_data_p[lbls_col]
            series_data_p = series_data_p.drop(lbls_col, axis=1)

            X_test_series_p = series_data_p.values.reshape((series_data_p.shape[0], X_test.shape[1], 1))

            subsequences = 2
            time_steps_p = X_test_series_p.shape[1] // subsequences
            X_test_series_sub_p = X_test_series_p.reshape((X_test_series_p.shape[0], subsequences, time_steps_p, 1))   

            cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub_p)

            train_p.loc[(train_p['Date'] == next_date.strftime("%Y-%m-%d"))  & (train_p['Weekly_Sales'] == 0), ['Weekly_Sales']] = cnn_lstm_test_prediction[0][0]
            predictions.loc[(predictions['Date'] == next_date.strftime("%Y-%m-%d")) & (predictions['Weekly_Sales'] == 0), ['Weekly_Sales']] = cnn_lstm_test_prediction[0][0]

        graph_data = actual_data.merge(predictions, left_on = ['Date'], right_on = ['Date'], how = 'outer')
        graph_data = graph_data.loc[graph_data['Date'] >= '2012-07-29']
        graph_data.loc[graph_data['Weekly_Sales_x'] == 0, ['Weekly_Sales_x']] = np.nan #Weekly_Sales_x is actual data and Weekly_Sales_y is the predicted data

        plt.subplots(figsize=(15, 6))
        plt.subplot(221)
        plt.plot(graph_data.Date, graph_data.Weekly_Sales_x, color='blue')
        plt.plot(graph_data.Date, graph_data.Weekly_Sales_y, dashes=[10, 5, 10, 5], color='red')
        plt.legend(['Actual Sales Data', 'Predicted Sales Data'], loc='upper right')
        plt.yticks(np.arange(0, 10, 2))
        plt.xticks(graph_data.Date, rotation=90)
        plt.title('Weekly sales and forecasts of Store-' + str(store) + ', Location-' + str(store_loc))
        plt.xlabel('Date (yyyy-mm-dd)')
        plt.ylabel('Weekly Sales (in 10 million dollars)')
        plt.grid(True)
    except:
        print("Insufficient data!")

    return plt.gcf()


In [9]:
def get_result_all():
    try:
        ########################################### sales data ##########################################################
        dataset = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\sales data-set.csv', usecols = ['Store', 'Item', 'Date', 'Weekly_Sales'])
        train = dataset[['Store', 'Item', 'Date', 'Weekly_Sales']]    
        #train = train.loc[train['Store'] == store]
        div = 10000000   #weekly sales in 100 million dollars
        file_name = "C:\\Users\\mrura\\Downloads\\MRP Final\\sales\\saved nn weights\\by store\\cnn_lstm_weights_store_all.hdf5"

        #Re-arranges the train dataset to apply shift methods
        train_r = train.sort_values('Date').groupby(['Item', 'Store', 'Date'], as_index=False)
        train_r = train_r.agg({'Weekly_Sales':['mean']})
        train_r.columns = ['Item', 'Store', 'Date', 'Weekly_Sales']
        train_r = train_r.groupby(['Date'], as_index=False)['Weekly_Sales'].sum()
        train_r['Weekly_Sales'] = train_r['Weekly_Sales'] / div

        #the model will use last 117 weekly sales data and 
        #current timestep (7 days) to forecast next weekly sales data 12 weeks ahead

        window = 117
        lag = 12

        series_data = time_series_data(train_r, window, lag)
        future_dates = series_data[['Date(t+%d)' % lag]]

        #drops Item and Store columns
        cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
        for i in range(window, 0, -1):
            cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

        series_data.drop(cols_to_drop, axis=1, inplace=True)
        series_data.drop(['Date(t)'], axis=1, inplace=True)

        lbls_col = 'Weekly_Sales(t+%d)' % lag
        lbls = series_data[lbls_col]
        series_data = series_data.drop(lbls_col, axis=1)

        X_train, X_test, Y_train, Y_test = train_test_split(series_data, lbls.values, test_size=0.3, random_state=0, shuffle=False)
        X_test = series_data     
        X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

        subsequences = 2
        time_steps = X_test_series.shape[1] // subsequences
        X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, time_steps, 1))
        X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, time_steps, 1))

        epochs = 2000
        batch = 32
        learning_rate = 0.00000001
        adam = optimizers.Adam(learning_rate)

        model_cnn_lstm = Sequential()
        model_cnn_lstm.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape = (None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
        model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model_cnn_lstm.add(TimeDistributed(Flatten()))
        model_cnn_lstm.add(LSTM(60, activation = 'relu'))
        model_cnn_lstm.add(Dropout(0.1))
        model_cnn_lstm.add(Dense(30, activation = 'relu'))
        model_cnn_lstm.add(Dropout(0.1))
        model_cnn_lstm.add(Dense(1))
        model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

        model_cnn_lstm.load_weights(file_name)
        model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

        #prediction
        #cnn_lstm_train_prediction = model_cnn_lstm.predict(X_train_series_sub)
        cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub)

        test_p = []
        for i in range(0, cnn_lstm_test_prediction.shape[0]):
            test_p.append(cnn_lstm_test_prediction[i][0])

        predictions = pd.DataFrame({'Date' : list(future_dates['Date(t+12)']), 'Weekly_Sales' : test_p})

        train_p = train_r
        actual_data = train_p.tail(117 + 12)
        #train_p.drop(['Item', 'Store'], axis=1, inplace=True)
        #actual_data.drop(['Item', 'Store'], axis=1, inplace=True)

        for k in range(0, 12):
            X_test_predict = train_p.tail(117 + 12)    
            max_date = X_test_predict['Date'].max()
            next_date = datetime.datetime.strptime(max_date, '%Y-%m-%d').date() + timedelta(days=7)
            df = pd.DataFrame({'Date' : next_date.strftime("%Y-%m-%d"), 
                               'Weekly_Sales' : [0]})
            X_test_predict = X_test_predict.append(df)
            train_p = train_p.append(df)
            predictions = predictions.append(df)
            actual_data = actual_data.append(df)

            X_test_predict = X_test_predict.reset_index(drop = True)
            train_p = train_p.reset_index(drop = True)
            predictions = predictions.reset_index(drop = True)
            actual_data = actual_data.reset_index(drop = True)

            series_data_p = time_series_data(X_test_predict, window, lag)

            cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
            for i in range(window, 0, -1):
                cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

            series_data_p.drop(cols_to_drop, axis=1, inplace=True)
            series_data_p.drop(['Date(t)'], axis=1, inplace=True)

            lbls_col = 'Weekly_Sales(t+%d)' % lag
            lbls = series_data_p[lbls_col]
            series_data_p = series_data_p.drop(lbls_col, axis=1)

            X_test_series_p = series_data_p.values.reshape((series_data_p.shape[0], X_test.shape[1], 1))

            subsequences = 2
            time_steps_p = X_test_series_p.shape[1] // subsequences
            X_test_series_sub_p = X_test_series_p.reshape((X_test_series_p.shape[0], subsequences, time_steps_p, 1))   

            cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub_p)

            train_p.loc[(train_p['Date'] == next_date.strftime("%Y-%m-%d"))  & (train_p['Weekly_Sales'] == 0), ['Weekly_Sales']] = cnn_lstm_test_prediction[0][0]
            predictions.loc[(predictions['Date'] == next_date.strftime("%Y-%m-%d")) & (predictions['Weekly_Sales'] == 0), ['Weekly_Sales']] = cnn_lstm_test_prediction[0][0]

        graph_data = actual_data.merge(predictions, left_on = ['Date'], right_on = ['Date'], how = 'outer')
        graph_data = graph_data.loc[graph_data['Date'] >= '2012-07-29']
        graph_data.loc[graph_data['Weekly_Sales_x'] == 0, ['Weekly_Sales_x']] = np.nan #Weekly_Sales_x is actual data and Weekly_Sales_y is the predicted data

        plt.subplots(figsize=(15, 6))
        plt.subplot(221)
        plt.plot(graph_data.Date, graph_data.Weekly_Sales_x, color='blue')
        plt.plot(graph_data.Date, graph_data.Weekly_Sales_y, dashes=[10, 5, 10, 5], color='red')
        plt.legend(['Actual Sales Data', 'Predicted Sales Data'], loc='upper right')
        plt.yticks(np.arange(0, 10, 2))
        plt.xticks(graph_data.Date, rotation=90)
        plt.title('Weekly sales and forecasts of all stores')
        plt.xlabel('Date (yyyy-mm-dd)')
        plt.ylabel('Weekly Sales (in 100 million dollars)')
        plt.grid(True)
    except:
        print("Insufficient data!")

    return plt.gcf()


In [10]:
def draw_fig(canvas, figure, loc=(0, 0)):    
    fig_canv_agg = FigureCanvasAgg(figure)
    fig_canv_agg.draw()
    fig_x, fig_y, fig_w, fig_h = figure.bbox.bounds
    fig_w, fig_h = int(fig_w), int(fig_h)
    img = Tk.PhotoImage(master=canvas, width=fig_w, height=fig_h)

    #set position
    canvas.create_image(loc[0] + fig_w/2, loc[1] + fig_h/2, image=img)
    tkagg.blit(img, fig_canv_agg.get_renderer()._renderer, colormode=2)

    return img


In [12]:
items = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\demand\\items.csv', usecols = ['item_code', 'item_name'])
item_name = items['item_name'].tolist()

stores = pd.read_csv('C:\\Users\\mrura\\Downloads\\MRP Final\\datasets\\demand\\stores_geo.csv', usecols = ['store_code'])
store_code = stores['store_code'].tolist()

sg.ChangeLookAndFeel('White')
fig_w, fig_h = 1000, 400

col_list = [[sg.Text('Select Store:', font=('current 12'))],
            [sg.Listbox(values=store_code, change_submits=True, size=(28, 10), key='store')],
            [sg.Text('Select Item:', font=('current 12'))],
            [sg.Listbox(values=item_name, change_submits=True, size=(28, 10), key='item')],
            [sg.Button('Show Result')]]

col_canv = sg.Column([[sg.Canvas(size=(fig_w, fig_h), key='canvas')]])

layout = [[sg.Column(col_list), sg.Pane([col_canv], size=(1024, 400))],]

window = sg.Window('Demands and Sales forecasts of the company',resizable=True, grab_anywhere=False).Layout(layout)
window.Finalize()

canv_elm = window.FindElement('canvas')

while True:
    event, values = window.Read()
    
    if event in (None, 'Exit'):
        break

    if event == 'Show Result':
        
        store = values['store'][0]
        store_loc = get_store_loc(store)
        if(store == 0):
            func = get_result_all
            item = 0
            item_name = ''
        else:
            func = get_result_by_item
            item = get_item_code(values['item'][0])
            item_name = values['item'][0]
            if(item == 0):
                func = get_result_by_store
        
        fig = func()
        fig_img = draw_fig(canv_elm.TKCanvas, fig)
           

** Warning - Look and Feel value not valid. Change your ChangeLookAndFeel call. **
